In [1]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline


In [2]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [3]:
train = pd.read_json("input/train.json")
#test = pd.read_json("input/test.json")

In [4]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target_train=train['is_iceberg']

In [ ]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [11]:

#X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=2,stratify = target_train, train_size=0.80)

### Cross Validation Folds

In [22]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=20,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches


In [23]:
def basic_cbamd_layer(X_input,filter_num =64,kernel_size = (3,3),conv_padding = 'valid',pool_size =(2,2),pool_strides = (2,2),pool_padding = 'valid',dropout_rate = 0.2):
    X = Conv2D(filter_num, kernel_size, padding = conv_padding)(X_input)
    #X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size,strides = pool_strides,padding = pool_padding)(X)
    X = Dropout(rate = dropout_rate)(X)
    return X

def dense_layer(X_input, units = 1,activation = 'relu',dropout_rate = 0.2):
    X = Dense(units = units,activation = activation)(X_input)
    X = Dropout(rate = dropout_rate)(X)
    return X
#kernel_regularizer=regularizers.l2(0.01)
def cnnmodel(input_shape,lr = 0.0001):
    X_input = Input(input_shape)
    #First composite layer
    X = basic_cbamd_layer(X_input,
                           filter_num =64,
                           kernel_size = (5,5),
                           conv_padding = 'valid',
                           pool_size =(3,3),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Second composite layer
    X = basic_cbamd_layer(X,
                           filter_num =128,
                           kernel_size = (3,3),
                           conv_padding = 'valid',
                           pool_size =(3,3),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Third composite layer
    X = basic_cbamd_layer(X,
                           filter_num =128,
                           kernel_size = (3,3),
                           conv_padding = 'valid',
                           pool_size =(2,2),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Forth composite layer
    X = basic_cbamd_layer(X,
                           filter_num =64,
                           kernel_size = (3,3),
                           conv_padding = 'valid',
                           pool_size =(3,3),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Flatten layer
    X = Flatten()(X)
    #First dense layer
    X = dense_layer(X, units = 256,activation = 'relu',dropout_rate = 0.2)
    #Second dense layer
    X = dense_layer(X, units = 128,activation = 'relu',dropout_rate = 0.2)
    #Decision layer
    X = dense_layer(X, units = 1,activation = 'sigmoid',dropout_rate = 0)
    
    model = Model(inputs=X_input,outputs=X)
    otimizer=Adam(lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='binary_crossentropy',
                  optimizer=otimizer,
                  metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7):
    #es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=False)
    return [msave]

#callbacks = get_callbacks(filepath=file_path, patience=3)

In [24]:
Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
def fitmodel(model,X_train,y_train,X_valid,y_valid,augment = False,epochs = 50,batch_size = 32,filepath = ".model_weights.hdf5"):
    if augment == False:
        result = model.fit(X_train, y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=(X_valid, y_valid),
                          callbacks=get_callbacks(filepath))
    else:
        result = model.fit_generator(data_augmentation(X_train,y_train,batch_size = batch_size),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=(X_valid, y_valid),
                          callbacks=get_callbacks(filepath))
    return result

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_17 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_18 (Activation)   (None, 33, 33, 128)       0         
__________

In [16]:
result_Cnn = fitmodel(Cnnmodel,X_train,target_train,X_train,target_train,augment = False,epochs = 200,batch_size = 32,filepath = 'model save/Model 2-Advanced CNN with local TL,DA and CV5/TL source.hdf5')

Train on 1604 samples, validate on 1604 samples
Epoch 1/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.7463 - acc: 0.5355 - val_loss: 0.6867 - val_acc: 0.5611
Epoch 2/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.7007 - acc: 0.5418 - val_loss: 0.6660 - val_acc: 0.6633
Epoch 3/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.6593 - acc: 0.5717 - val_loss: 0.6125 - val_acc: 0.6739
Epoch 4/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.5984 - acc: 0.6365 - val_loss: 0.5647 - val_acc: 0.6802
Epoch 5/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.5771 - acc: 0.6471 - val_loss: 0.5454 - val_acc: 0.6939
Epoch 6/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.5478 - acc: 0.6808 - val_loss: 0.5299 - val_acc: 0.7101
Epoch 7/200
1604/1604 [==============================] - 9s 5ms/step - loss: 0.5347 - acc: 0.6951 - val_loss: 0.5218 - val_acc: 0.7400
Epoch 8

Epoch 61/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.1941 - acc: 0.9077 - val_loss: 0.1802 - val_acc: 0.9146
Epoch 62/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.1973 - acc: 0.9146 - val_loss: 0.1469 - val_acc: 0.9426
Epoch 63/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.1857 - acc: 0.9152 - val_loss: 0.1390 - val_acc: 0.9401
Epoch 64/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.1997 - acc: 0.9127 - val_loss: 0.1445 - val_acc: 0.9408
Epoch 65/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.1755 - acc: 0.9264 - val_loss: 0.1466 - val_acc: 0.9358
Epoch 66/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.1654 - acc: 0.9183 - val_loss: 0.1283 - val_acc: 0.9439
Epoch 67/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.1782 - acc: 0.9171 - val_loss: 0.1594 - val_acc: 0.9289
Epoch 68/200
1604/1604 [========================

1604/1604 [==============================] - 8s 5ms/step - loss: 0.0889 - acc: 0.9670 - val_loss: 0.0467 - val_acc: 0.9882
Epoch 122/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0795 - acc: 0.9626 - val_loss: 0.0462 - val_acc: 0.9825
Epoch 123/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.1003 - acc: 0.9570 - val_loss: 0.0423 - val_acc: 0.9869
Epoch 124/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0782 - acc: 0.9670 - val_loss: 0.0404 - val_acc: 0.9869
Epoch 125/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0902 - acc: 0.9645 - val_loss: 0.0418 - val_acc: 0.9894
Epoch 126/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0826 - acc: 0.9676 - val_loss: 0.0416 - val_acc: 0.9869
Epoch 127/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0900 - acc: 0.9620 - val_loss: 0.0392 - val_acc: 0.9913
Epoch 128/200
1604/1604 [==============================

Epoch 181/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0401 - acc: 0.9863 - val_loss: 0.0170 - val_acc: 0.9969
Epoch 182/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0410 - acc: 0.9875 - val_loss: 0.0085 - val_acc: 0.9994
Epoch 183/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0535 - acc: 0.9825 - val_loss: 0.0186 - val_acc: 0.9938
Epoch 184/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0311 - acc: 0.9894 - val_loss: 0.0134 - val_acc: 0.9963
Epoch 185/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0680 - acc: 0.9688 - val_loss: 0.0238 - val_acc: 0.9963
Epoch 186/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0506 - acc: 0.9788 - val_loss: 0.0097 - val_acc: 0.9988
Epoch 187/200
1604/1604 [==============================] - 8s 5ms/step - loss: 0.0317 - acc: 0.9888 - val_loss: 0.0122 - val_acc: 0.9981
Epoch 188/200
1604/1604 [================

In [17]:
def save_history(result,name = 'fit_history.csv',path = os.getcwd()):
    train_loss = result.history['loss']
    val_loss = result.history['val_loss']
    train_accuracy = result.history['acc']
    val_accuracy = result.history['val_acc']
    fit_history = pd.DataFrame({'train_loss':train_loss,'val_loss':val_loss,'train_accuracy':train_accuracy,'val_accuracy':val_accuracy})
    fit_history.to_csv(os.path.join(path,name))

In [137]:
save_history(result_Cnn)

NameError: name 'result_Cnn' is not defined

In [26]:
Train_cv = StratifiedKFold(n_splits=5,shuffle = True,random_state=2).split(X_train,target_train)
cv_val_list = []
for i,masks in enumerate(Train_cv):
    train_mask,valid_mask = masks
    X_train_cv = X_train[train_mask]
    y_train_cv = target_train[train_mask]
    X_valid = X_train[valid_mask]
    y_valid = target_train[valid_mask]
    path = 'model save/Model 2-Advanced CNN with local TL,DA and CV5/'
    modelseries = '.cv'+str(i+1)+'.hdf5'
    modelpath = os.path.join(path,modelseries)
    Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
    Cnnmodel.load_weights('model save/Model 2-Advanced CNN with local TL,DA and CV5/TL source.hdf5')
    result_Cnn = fitmodel(Cnnmodel,X_train_cv,y_train_cv,X_valid,y_valid,augment = True,epochs = 100,batch_size = 32,filepath = modelpath)
    historyseries = 'cv'+str(i+1)+'.csv'
    save_history(result_Cnn,name = historyseries,path = path)
    Cnnmodel.load_weights(filepath=modelpath)
    score = Cnnmodel.evaluate(X_valid, y_valid, verbose=1)
    cv_val_list.append(score)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_25 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_43 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_26 (Activation)   (None, 33, 33, 128)       0         
__________

41/40 [==============================] - 6s 153ms/step - loss: 0.2506 - acc: 0.9092 - val_loss: 0.1252 - val_acc: 0.9720
Epoch 37/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2411 - acc: 0.9047 - val_loss: 0.1046 - val_acc: 0.9627
Epoch 38/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2139 - acc: 0.9085 - val_loss: 0.0966 - val_acc: 0.9627
Epoch 39/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2273 - acc: 0.9001 - val_loss: 0.0931 - val_acc: 0.9689
Epoch 40/100
41/40 [==============================] - 6s 134ms/step - loss: 0.2311 - acc: 0.9123 - val_loss: 0.1012 - val_acc: 0.9783
Epoch 41/100
41/40 [==============================] - 5s 133ms/step - loss: 0.2228 - acc: 0.9009 - val_loss: 0.0959 - val_acc: 0.9845
Epoch 42/100
41/40 [==============================] - 6s 138ms/step - loss: 0.2274 - acc: 0.8978 - val_loss: 0.1105 - val_acc: 0.9720
Epoch 43/100
41/40 [==============================] - 6s 137ms/step - loss:

41/40 [==============================] - 5s 133ms/step - loss: 0.2340 - acc: 0.9034 - val_loss: 0.1408 - val_acc: 0.9472
Epoch 98/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2175 - acc: 0.9161 - val_loss: 0.1257 - val_acc: 0.9596
Epoch 99/100
41/40 [==============================] - 6s 137ms/step - loss: 0.2784 - acc: 0.8836 - val_loss: 0.1694 - val_acc: 0.9317
Epoch 100/100
322/322 [==============================] - 0s 1ms/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_29 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_

Epoch 32/100
41/40 [==============================] - 5s 134ms/step - loss: 0.3035 - acc: 0.8662 - val_loss: 0.1299 - val_acc: 0.9439
Epoch 33/100
41/40 [==============================] - 6s 136ms/step - loss: 0.2541 - acc: 0.8982 - val_loss: 0.1298 - val_acc: 0.9377
Epoch 34/100
41/40 [==============================] - 5s 133ms/step - loss: 0.2376 - acc: 0.8932 - val_loss: 0.1573 - val_acc: 0.9408
Epoch 35/100
41/40 [==============================] - 6s 152ms/step - loss: 0.2661 - acc: 0.8799 - val_loss: 0.1293 - val_acc: 0.9533
Epoch 36/100
41/40 [==============================] - 5s 133ms/step - loss: 0.2481 - acc: 0.8963 - val_loss: 0.1337 - val_acc: 0.9470
Epoch 37/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2510 - acc: 0.8830 - val_loss: 0.1270 - val_acc: 0.9377
Epoch 38/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2290 - acc: 0.9031 - val_loss: 0.1278 - val_acc: 0.9439
Epoch 39/100
41/40 [==============================] - 6s 137ms

41/40 [==============================] - 5s 134ms/step - loss: 0.1980 - acc: 0.9184 - val_loss: 0.1500 - val_acc: 0.9377
Epoch 94/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2010 - acc: 0.9184 - val_loss: 0.1545 - val_acc: 0.9377
Epoch 95/100
41/40 [==============================] - 6s 148ms/step - loss: 0.1845 - acc: 0.9199 - val_loss: 0.1823 - val_acc: 0.9159
Epoch 96/100
41/40 [==============================] - 6s 142ms/step - loss: 0.1761 - acc: 0.9298 - val_loss: 0.1919 - val_acc: 0.9097
Epoch 97/100
41/40 [==============================] - 6s 140ms/step - loss: 0.2065 - acc: 0.9176 - val_loss: 0.1601 - val_acc: 0.9252
Epoch 98/100
41/40 [==============================] - 6s 136ms/step - loss: 0.1805 - acc: 0.9298 - val_loss: 0.1366 - val_acc: 0.9346
Epoch 99/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2228 - acc: 0.9059 - val_loss: 0.1834 - val_acc: 0.9190
Epoch 100/100
321/321 [==============================] - 0s 1ms/step
______

...

41/40 [==============================] - 5s 133ms/step - loss: 0.1931 - acc: 0.9207 - val_loss: 0.1124 - val_acc: 0.9626
Epoch 90/100
41/40 [==============================] - 6s 135ms/step - loss: 0.1984 - acc: 0.9154 - val_loss: 0.1543 - val_acc: 0.9470
Epoch 91/100
41/40 [==============================] - 6s 137ms/step - loss: 0.1951 - acc: 0.9192 - val_loss: 0.1362 - val_acc: 0.9470
Epoch 92/100
41/40 [==============================] - 6s 137ms/step - loss: 0.1872 - acc: 0.9253 - val_loss: 0.1384 - val_acc: 0.9533
Epoch 93/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2013 - acc: 0.9291 - val_loss: 0.1305 - val_acc: 0.9502
Epoch 94/100
41/40 [==============================] - 5s 133ms/step - loss: 0.2076 - acc: 0.9146 - val_loss: 0.1199 - val_acc: 0.9688
Epoch 95/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2138 - acc: 0.9176 - val_loss: 0.1377 - val_acc: 0.9502
Epoch 96/100
41/40 [==============================] - 6s 135ms/step - loss:

Epoch 24/100
41/40 [==============================] - 6s 137ms/step - loss: 0.2551 - acc: 0.8925 - val_loss: 0.1273 - val_acc: 0.9469
Epoch 25/100
41/40 [==============================] - 5s 133ms/step - loss: 0.2361 - acc: 0.8963 - val_loss: 0.1031 - val_acc: 0.9656
Epoch 26/100
41/40 [==============================] - 6s 137ms/step - loss: 0.2200 - acc: 0.9024 - val_loss: 0.1203 - val_acc: 0.9469
Epoch 27/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2616 - acc: 0.8925 - val_loss: 0.1028 - val_acc: 0.9719
Epoch 28/100
41/40 [==============================] - 7s 177ms/step - loss: 0.2535 - acc: 0.8971 - val_loss: 0.1105 - val_acc: 0.9719
Epoch 29/100
41/40 [==============================] - 6s 154ms/step - loss: 0.2495 - acc: 0.8948 - val_loss: 0.1059 - val_acc: 0.9750
Epoch 30/100
41/40 [==============================] - 6s 140ms/step - loss: 0.2588 - acc: 0.9025 - val_loss: 0.1069 - val_acc: 0.9719
Epoch 31/100
41/40 [==============================] - 6s 144ms

41/40 [==============================] - 5s 133ms/step - loss: 0.2113 - acc: 0.9063 - val_loss: 0.1703 - val_acc: 0.9375
Epoch 86/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2296 - acc: 0.8918 - val_loss: 0.1179 - val_acc: 0.9563
Epoch 87/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2083 - acc: 0.9138 - val_loss: 0.1333 - val_acc: 0.9625
Epoch 88/100
41/40 [==============================] - 5s 132ms/step - loss: 0.2179 - acc: 0.9077 - val_loss: 0.1149 - val_acc: 0.9656
Epoch 89/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2193 - acc: 0.8949 - val_loss: 0.1346 - val_acc: 0.9656
Epoch 90/100
41/40 [==============================] - 5s 131ms/step - loss: 0.1967 - acc: 0.9237 - val_loss: 0.1048 - val_acc: 0.9656
Epoch 91/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2218 - acc: 0.9004 - val_loss: 0.1403 - val_acc: 0.9500
Epoch 92/100
41/40 [==============================] - 5s 133ms/step - loss:

Epoch 20/100
41/40 [==============================] - 6s 151ms/step - loss: 0.2341 - acc: 0.9032 - val_loss: 0.1047 - val_acc: 0.9688
Epoch 21/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2226 - acc: 0.9039 - val_loss: 0.1025 - val_acc: 0.9719
Epoch 22/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2638 - acc: 0.8955 - val_loss: 0.1461 - val_acc: 0.9531
Epoch 23/100
41/40 [==============================] - 6s 137ms/step - loss: 0.2400 - acc: 0.9054 - val_loss: 0.1083 - val_acc: 0.9719
Epoch 24/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2465 - acc: 0.9054 - val_loss: 0.1164 - val_acc: 0.9719
Epoch 25/100
41/40 [==============================] - 5s 133ms/step - loss: 0.2434 - acc: 0.9001 - val_loss: 0.1317 - val_acc: 0.9594
Epoch 26/100
41/40 [==============================] - 6s 144ms/step - loss: 0.2290 - acc: 0.8971 - val_loss: 0.1227 - val_acc: 0.9500
Epoch 27/100
41/40 [==============================] - 5s 131ms

41/40 [==============================] - 5s 132ms/step - loss: 0.1969 - acc: 0.9199 - val_loss: 0.1106 - val_acc: 0.9625
Epoch 82/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2164 - acc: 0.9124 - val_loss: 0.1352 - val_acc: 0.9531
Epoch 83/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2080 - acc: 0.9108 - val_loss: 0.1436 - val_acc: 0.9437
Epoch 84/100
41/40 [==============================] - 5s 132ms/step - loss: 0.2135 - acc: 0.9123 - val_loss: 0.1208 - val_acc: 0.9625
Epoch 85/100
41/40 [==============================] - 5s 134ms/step - loss: 0.2139 - acc: 0.9109 - val_loss: 0.1367 - val_acc: 0.9531
Epoch 86/100
41/40 [==============================] - 5s 132ms/step - loss: 0.2134 - acc: 0.9100 - val_loss: 0.1604 - val_acc: 0.9531
Epoch 87/100
41/40 [==============================] - 6s 135ms/step - loss: 0.2021 - acc: 0.9231 - val_loss: 0.1066 - val_acc: 0.9531
Epoch 88/100
41/40 [==============================] - 5s 134ms/step - loss:

In [27]:
cv_val_list

[[0.14604590879464002, 0.92546583850931674],
 [0.14806677773304724, 0.9190031152647975],
 [0.10787337619196218, 0.96573208722741433],
 [0.10974582657217979, 0.96562499999999996],
 [0.10343162231147289, 0.96875]]

In [143]:
test = pd.read_csv('fit_history.csv')

In [147]:
name = 'cv1.csv'
path ='model save/Model 1-Advanced CNN with DA and CV5/'
test.to_csv(os.path.join(path,name))